## U.S. Bureau of Labor Statistics - CPI Analysis
#### Eric Bottinelli

### 1. Retrieve data via BLS API v2

**Documentation**

- https://www.bls.gov/developers/api_python.htm
- https://data.bls.gov/cgi-bin/surveymost?cu
- https://data.bls.gov/dataQuery/find?fq=survey:[cu]&s=popularity:D&r=100&st=0
- https://www.bls.gov/cpi/tables/relative-importance/2023.htm
- https://www.bls.gov/bls/news-release/cpi.htm

**Packages to install**

- Prettytable ('pip install prettytable')

**API Series ID**

Consumer Price Index for All Urban Consumers (CPI-U)
- *All items in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0
    - SA: CUSR0000SA0
- *All items less food and energy in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0L1E
    - SA: CUSR0000SA0L1E
- *Food in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAF1
    - SA: CUSR0000SAF1
- *Food at home in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAF11
    - SA: CUSR0000SAF11
- *Energy in U.S. city average, all urban consumers*
    - NSA: CUUR0000SA0E
    - SA: CUSR0000SA0E
- *Commodities less food and energy commodities in U.S. city average, all urban consumers*
    - NSA: CUUR0000SACL1E
    - SA: CUSR0000SACL1E
- *Services less energy services in U.S. city average, all urban consumers*
    - NSA: CUUR0000SASLE
    - SA: CUSR0000SASLE
- *Shelter in U.S. city average, all urban consumers*
    - NSA: CUUR0000SAH1
    - SA: CUSR0000SAH1
((https://www.bls.gov/cpi/factsheets/owners-equivalent-rent-and-rent.htm))

**Calculate special CPI**

Occasionally, a user wishes to estimate a price change that is not published by BLS. For instance, suppose a user would like a CPI series for ‘services less energy services and shelter’. This can be done by estimating a special index, in this case, ‘services less energy services and shelter’.
[BLS Doc](https://www.bls.gov/cpi/factsheets/constructing-special-cpis.htm)

If SEEB01 -> CUUR0000SEEB01

Cost weight is just a sum of all the items

If I add all the values to calculate the services less energy services and shelter, it becomes a lot of data. Explore different solution (e.g. remove goods from core CPI)

**Supercore CPI**

"Fed Chair Jerome Powell cited a specific category of inflation—inflation in core services other than housing—as being perhaps “the most important category for understanding the future evolution of core inflation.” The financial press has termed this category “supercore” inflation" ([FED of St. Louis](https://www.stlouisfed.org/on-the-economy/2024/may/measuring-inflation-headline-core-supercore-services))

In [15]:
import os
import requests
import json
import prettytable
import pandas as pd
from datetime import datetime

folder_name = 'CPI_Data'

food_weight = 13.555
energy_weight = 6.655
services_less_energy_weight = 60.899
shelter_weight = 36.191

weight_map = {
    'All_Items': 100,
    'Food_Energy': food_weight + energy_weight,
    'Food': food_weight,
    'Food_At_Home': 8.167,
    'Food_Away_From_Home': 5.388,
    'Energy': energy_weight,
    'All_Items_Less_Food_Energy': 79.790,
    'Commodities_Less_Food_Energy_Commodities': 18.891,
    'Services_Less_Energy_Services': services_less_energy_weight,
    'Shelter': shelter_weight,
    'Supercore': services_less_energy_weight - shelter_weight 
}

In [16]:
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

current_date = datetime.now()
current_year = current_date.year
last_year = current_year - 1

series_names = {
    'CUUR0000SA0': 'NSA_All_Items',
    'CUSR0000SA0': 'SA_All_Items',
    'CUUR0000SA0L1E': 'NSA_All_Items_Less_Food_Energy',
    'CUSR0000SA0L1E': 'SA_All_Items_Less_Food_Energy',
    'CUUR0000SAF': 'NSA_Food',
    'CUSR0000SAF': 'SA_Food',
    'CUUR0000SAF11': 'NSA_Food_At_Home',
    'CUSR0000SAF11': 'SA_Food_At_Home',
    'CUUR0000SEFV': 'NSA_Food_Away_From_Home',
    'CUSR0000SEFV': 'SA_Food_Away_From_Home',
    'CUUR0000SA0E': 'NSA_Energy',
    'CUSR0000SA0E': 'SA_Energy',
    'CUUR0000SACL1E': 'NSA_Commodities_Less_Food_Energy_Commodities',
    'CUSR0000SACL1E': 'SA_Commodities_Less_Food_Energy_Commodities',
    'CUUR0000SASLE': 'NSA_Services_Less_Energy_Services',
    'CUSR0000SASLE': 'SA_Services_Less_Energy_Services',
    'CUUR0000SAH1': 'NSA_Shelter',
    'CUSR0000SAH1': 'SA_Shelter',
}
series_ids = list(series_names.keys())

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": series_ids, "startyear": str(last_year), "endyear": str(current_year)})
response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(response.text)

all_data = []
for series in json_data['Results']['series']:
    rows = []
    for item in series['data']:
        footnotes = "".join([footnote['text'] + ',' for footnote in item['footnotes'] if footnote]).rstrip(',')
        if 'M01' <= item['period'] <= 'M12':
            rows.append([series_names[series['seriesID']], item['year'], item['period'], item['value'], footnotes])

    df = pd.DataFrame(rows, columns=["series id", "year", "period", "value", "footnotes"])
    all_data.append(df)

complete_data = pd.concat(all_data)

df = complete_data.copy()
df['date'] = pd.to_datetime(df['year'].astype(str) + df['period'].str.replace('M', ''), format='%Y%m')
df['series id'] = df['series id'].astype(str) 
df['value'] = pd.to_numeric(df['value'], errors='coerce')
df['footnotes'] = df['footnotes'].astype(str) 
df.drop(['year', 'period', 'footnotes'], axis=1, inplace=True)
df.rename(columns={'series id': 'id'}, inplace=True)
df = df[['id', 'date', 'value']]

csv_path = os.path.join(folder_name, 'CPI_data.csv')
df.to_csv(csv_path, index=False)

In [21]:
df = pd.read_csv("CPI_Data/CPI_data.csv")
df['date'] = pd.to_datetime(df['date'])

In [22]:
df['MoM_change'] = df.groupby('id')['value'].transform(lambda x: (x - x.shift(-1)) / x.shift(-1))
df['YoY_change'] = df.groupby('id')['value'].transform(lambda x: (x - x.shift(-12)) / x.shift(-12))

In [23]:
def calculate_weighted_change(df, id1, id2, new_id, weight1, weight2, weight_total, operation): 
    series1 = df.loc[df['id'] == id1, ['MoM_change', 'YoY_change']].set_index(df.loc[df['id'] == id1, 'date']) * weight1
    series2 = df.loc[df['id'] == id2, ['MoM_change', 'YoY_change']].set_index(df.loc[df['id'] == id2, 'date']) * weight2

    if operation == 'add':
        result = (series1 + series2) / weight_total
    elif operation == 'subtract':
        result = (series1 - series2) / weight_total
    else:
        raise ValueError("Operation must be 'add' or 'subtract'.")

    result = result.reset_index()
    result['id'] = new_id
    result['value'] = 0

    result = result[['id', 'date', 'value', 'MoM_change', 'YoY_change']]
    return result

# Calculate 'SA_Food_Energy' and 'NSA_Food_Energy'
df = pd.concat([
    df,
    calculate_weighted_change(df, 'SA_Food', 'SA_Energy', 'SA_Food_Energy', weight_map['Food'], weight_map['Energy'], weight_map['Food_Energy'], 'add'),
    calculate_weighted_change(df, 'NSA_Food', 'NSA_Energy', 'NSA_Food_Energy', weight_map['Food'], weight_map['Energy'], weight_map['Food_Energy'], 'add')
])

# Calculate 'SA_Supercore' and 'NSA_Supercore'
df = pd.concat([
    df,
    calculate_weighted_change(df, 'SA_Services_Less_Energy_Services', 'SA_Shelter', 'SA_Supercore', weight_map['Services_Less_Energy_Services'], weight_map['Shelter'], weight_map['Supercore'], 'subtract'),
    calculate_weighted_change(df, 'NSA_Services_Less_Energy_Services', 'NSA_Shelter', 'NSA_Supercore', weight_map['Services_Less_Energy_Services'], weight_map['Shelter'], weight_map['Supercore'], 'subtract')
])

In [24]:
category_map = {
    'All_Items': (0, 'Headline', '', '', 'All Items'),
    'Food_Energy': (1, 'Food + Energy', '', '', 'Food and Energy'),
    'Food': (2, '', 'Food', '', 'Food'),
    'Food_At_Home': (3, '', '', 'At home', 'Food at Home'),
    'Food_Away_From_Home': (4, '', '', 'Away Home', 'Food Away from Home'),
    'Energy': (5, '', 'Energy', '', 'Energy'),
    'All_Items_Less_Food_Energy': (6, 'Core', '', '', 'Core CPI'),
    'Commodities_Less_Food_Energy_Commodities': (7, '', 'Goods', '', 'Goods'),
    'Services_Less_Energy_Services': (8, '', 'Services', '', 'Services excluding Energy'),
    'Shelter': (9, '', '', 'Shelter', 'Shelter'),
    'Supercore': (10, '', '', 'Supercore', 'Supercore')
}

ordered_categories = ['Headline', 'Food + Energy', 'Core']
ordered_sub_categories_1 = ['Food', 'Energy', 'Commodities', 'Services']
ordered_sub_categories_2 = ['At home', 'Away Home', 'Shelter', 'Supercore']

for i in range(4):
    if i in [0, 1]:
        data = df[df['id'].str.startswith('NSA_')].copy()
        prefix_length = 4
    else:
        data = df[df['id'].str.startswith('SA_')].copy() 
        prefix_length = 3

    data.loc[:, 'id'] = data['id'].str[prefix_length:]

    missing_ids = set(data['id']) - set(category_map.keys())
    if missing_ids:
        raise ValueError(f"Missing IDs in category_map: {missing_ids}")

    data.loc[:, 'Month-Year'] = data['date'].dt.strftime('%b-%y')
    order_cat = data['id'].apply(lambda x: category_map.get(x, (None, None, None, None, None)))
    data.loc[:, 'Order'] = [item[0] for item in order_cat]
    data.loc[:, 'Category'] = [item[1] for item in order_cat]
    data.loc[:, 'Sub Category 1'] = [item[2] for item in order_cat]
    data.loc[:, 'Sub Category 2'] = [item[3] for item in order_cat]
    data.loc[:, 'Name'] = [item[4] for item in order_cat]
    data.loc[:, 'Weight'] = data['id'].map(weight_map).fillna('Unknown')

    if i in [0, 2]:
        value_column = 'MoM_change'
    else:
        value_column = 'YoY_change'

    data_pivot = data.pivot_table(
        index=['Name', 'Order', 'Category', 'Sub Category 1', 'Sub Category 2', 'Weight'],
        columns='Month-Year',
        values=value_column,
        aggfunc='first'
    )

    data_pivot = data_pivot[sorted(data_pivot.columns, key=lambda x: pd.to_datetime(x, format='%b-%y'), reverse=True)]
    data_pivot.columns.name = None
    data_pivot.reset_index(inplace=True)
    data_pivot.sort_values(by='Order', inplace=True)
    data_pivot.drop(columns=['Order'], inplace=True)

    if i == 0:
        NSA_MoM_df = data_pivot
    elif i == 1:
        NSA_YoY_df = data_pivot
    elif i == 2:
        SA_MoM_df = data_pivot
    else:
        SA_YoY_df = data_pivot

In [25]:
csv_path = os.path.join(folder_name, 'NSA_MoM_CPI_data.csv')
NSA_MoM_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'NSA_YoY_CPI_data.csv')
NSA_YoY_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'SA_MoM_CPI_data.csv')
SA_MoM_df.to_csv(csv_path, index=False)

csv_path = os.path.join(folder_name, 'SA_YoY_CPI_data.csv')
SA_YoY_df.to_csv(csv_path, index=False)

In [24]:
import requests
from bs4 import BeautifulSoup

def fetch_report_text(url):
    headers = {
        'User-Agent': 'youremail@domain.name'  # Replace with your actual email
    }
    
    # Make the GET request
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the <div> with class 'normalnews'
        normalnews_div = soup.find('div', class_='normalnews')
        
        if normalnews_div:
            # Extract text from the <div>
            return normalnews_div.get_text(separator='\n', strip=True)
        else:
            return "No <div> with class 'normalnews' found."
    else:
        return f"Failed to retrieve page: {response.status_code}"

url = "https://www.bls.gov/news.release/cpi.nr0.htm"

# Fetch the full text from the page
full_text = fetch_report_text(url)

Full Text Retrieved:
 Transmission of material in this release is embargoed until                                       
8:30 a.m. (ET) Wednesday, August 14, 2024      USDL-24-1662
	
Technical information: (202) 691-7000  *  cpi_info@bls.gov  *  www.bls.gov/cpi
Media contact:         (202) 691-5902  *  PressOffice@bls.gov 

CONSUMER PRICE INDEX - JULY 2024

The Consumer Price Index for All Urban Consumers (CPI-U) increased 0.2 percent on a seasonally adjusted
basis, after declining 0.1 percent in June, the U.S. Bureau of Labor Statistics reported today. Over the
last 12 months, the all items index increased 2.9 percent before seasonal adjustment.

The index for shelter rose 0.4 percent in July, accounting for nearly 90 percent of the monthly increase
in the all items index. The energy index was unchanged over the month, after declining in the two
preceding months. The index for food increased 0.2 percent in July, as it did in June. The food away from
home index rose 0.2 percent over th

In [25]:
# OLD METHOD: KEYWORDS

# def extract_relevant_paragraph(full_text):
#     """
#     Extracts the relevant paragraph from the full text based on the presence of specific keywords.
    
#     Args:
#         full_text (str): The complete text content inside the <div class="normalnews"> tag.
    
#     Returns:
#         str: The relevant paragraph containing information about shelter, energy, and food.
#     """
#     # Split the text into paragraphs using two new lines (or other specific delimiters based on the source text)
#     paragraphs = full_text.split('\n\n')
    
#     # Keywords to identify the paragraph of interest
#     keywords = ["shelter", "energy", "food", "food at home", "food away from home"]
    
#     # Iterate over paragraphs and find one that contains any of the keywords
#     for paragraph in paragraphs:
#         # Combine lines to ensure full paragraph is considered
#         paragraph_combined = " ".join(paragraph.splitlines())
#         if any(keyword in paragraph_combined.lower() for keyword in keywords):
#             return paragraph_combined
    
#     return "No relevant paragraph found with the specified keywords."


In [33]:
def extract_paragraph_based_on_index(full_text):
    # Normalize newlines and split the text into paragraphs
    paragraphs = [p.strip() for p in full_text.replace('\r', '').split('\n\n') if p.strip()]
    
    # Locate the starting point for the relevant content
    for i, paragraph in enumerate(paragraphs):
        if "CONSUMER PRICE INDEX" in paragraph:
            start_index = i
            break
    else:
        return "CONSUMER PRICE INDEX section not found."

    # Check if a "NOTE:" is present immediately after "CONSUMER PRICE INDEX"
    note_present = "NOTE:" in paragraphs[start_index + 1]

    # Determine the target paragraph
    target_index = start_index + 3 if note_present else start_index + 2

    if target_index < len(paragraphs):
        # Get the target paragraph and replace newline characters with a space
        target_paragraph = paragraphs[target_index].replace('\n', ' ')
        return target_paragraph
    else:
        return "The paragraph of interest could not be found."


In [35]:
full_text= """
Consumer Price Index News Release
Transmission of material in this release is embargoed until                                        
8:30 a.m. (ET) Thursday, July 11, 2024         USDL-24-1325
	
Technical information: (202) 691-7000  *  cpi_info@bls.gov  *  www.bls.gov/cpi
Media contact:         (202) 691-5902  *  PressOffice@bls.gov 

CONSUMER PRICE INDEX - JUNE 2024

(NOTE: This news release was reissued on July 11, 2024. BLS inadvertently published an index value and
related 1-month and 12-month percent changes for inpatient hospital services for June 2024 that did not
meet publication criteria. These data have been removed from tables 2, 6, and 7 of the news release.
These data were not published in the database.) 

The Consumer Price Index for All Urban Consumers (CPI-U) declined 0.1 percent on a seasonally adjusted
basis, after being unchanged in May, the U.S. Bureau of Labor Statistics reported today. Over the last 12
months, the all items index increased 3.0 percent before seasonal adjustment.

The index for gasoline fell 3.8 percent in June, after declining 3.6 percent in May, more than offsetting
an increase in shelter. The energy index fell 2.0 percent over the month, as it did the preceding month.
The index for food increased 0.2 percent in June. The food away from home index rose 0.4 percent over the
month, while the food at home index increased 0.1 percent. 

The index for all items less food and energy rose 0.1 percent in June, after rising 0.2 percent the
preceding month. Indexes which increased in June include shelter, motor vehicle insurance, household
furnishings and operations, medical care, and personal care. The indexes for airline fares, used cars and
trucks, and communication were among those that decreased over the month.

The all items index rose 3.0 percent for the 12 months ending June, a smaller increase than the
3.3-percent increase for the 12 months ending May. The all items less food and energy index rose 3.3
percent over the last 12 months and was the smallest 12-month increase in that index since April 2021.
The energy index increased 1.0 percent for the 12 months ending June. The food index increased 2.2
percent over the last year. 

Table A. Percent changes in CPI for All Urban Consumers (CPI-U): U.S. city average

Seasonally adjusted changes from preceding month	Un-
adjusted
12-mos.
ended
Jun. 2024
Dec.
2023	Jan.
2024	Feb.
2024	Mar.
2024	Apr.
2024	May
2024	Jun.
2024
All items

0.2	0.3	0.4	0.4	0.3	0.0	-0.1	3.0
Food

0.2	0.4	0.0	0.1	0.0	0.1	0.2	2.2
Food at home

0.1	0.4	0.0	0.0	-0.2	0.0	0.1	1.1
Food away from home(1)

0.3	0.5	0.1	0.3	0.3	0.4	0.4	4.1
Energy

-0.2	-0.9	2.3	1.1	1.1	-2.0	-2.0	1.0
Energy commodities

-0.7	-3.2	3.6	1.5	2.7	-3.5	-3.7	-2.2
Gasoline (all types)

-0.6	-3.3	3.8	1.7	2.8	-3.6	-3.8	-2.5
Fuel oil

-3.3	-4.5	1.1	-1.3	0.9	-0.4	-2.4	0.8
Energy services

0.3	1.4	0.8	0.7	-0.7	-0.2	-0.1	4.3
Electricity

0.6	1.2	0.3	0.9	-0.1	0.0	-0.7	4.4
Utility (piped) gas service

-0.6	2.0	2.3	0.0	-2.9	-0.8	2.4	3.7
All items less food and energy

0.3	0.4	0.4	0.4	0.3	0.2	0.1	3.3
Commodities less food and energy commodities

-0.1	-0.3	0.1	-0.2	-0.1	0.0	-0.1	-1.8
New vehicles

0.2	0.0	-0.1	-0.2	-0.4	-0.5	-0.2	-0.9
Used cars and trucks

0.6	-3.4	0.5	-1.1	-1.4	0.6	-1.5	-10.1
Apparel

0.0	-0.7	0.6	0.7	1.2	-0.3	0.1	0.8
Medical care commodities(1)

-0.1	-0.6	0.1	0.2	0.4	1.3	0.2	3.1
Services less energy services

0.4	0.7	0.5	0.5	0.4	0.2	0.1	5.1
Shelter

0.4	0.6	0.4	0.4	0.4	0.4	0.2	5.2
Transportation services

0.1	1.0	1.4	1.5	0.9	-0.5	-0.5	9.4
Medical care services

0.5	0.7	-0.1	0.6	0.4	0.3	0.2	3.3
Footnotes
(1) Not seasonally adjusted.
Food

The food index rose 0.2 percent in June after increasing 0.1 percent in May. The index for food at home
rose 0.1 percent in June. Four of the six major grocery store food group indexes increased over the month.
The index for other food at home rose 0.5 percent in June as the index for butter and margarine increased
2.4 percent. The dairy and related products index rose 0.6 percent over the month and the meats, poultry,
fish, and eggs index increased 0.2 percent. The index for nonalcoholic beverages rose 0.1 percent in June.

The index for fruits and vegetables fell 0.5 percent in June, after being unchanged in May. The cereals
and bakery products index decreased 0.1 percent over the month. 

The food away from home index rose 0.4 percent in June, as it did in May. The index for full service
meals rose 0.6 percent and the index for limited service meals increased 0.2 percent over the month.

The food at home index rose 1.1 percent over the last 12 months. The index for meats, poultry, fish, and
eggs rose 2.6 percent over the last 12 months and the index for other food at home increased 1.6 percent.
Over the same period, the nonalcoholic beverages index rose 1.5 percent and the cereals and bakery
products index increased 0.5 percent. In comparison, the fruits and vegetables index fell 0.5 percent
over the year and the dairy and related products index declined 0.1 percent. 

The index for food away from home rose 4.1 percent over the last year. The index for limited service meals
rose 4.3 percent over the last 12 months and the index for full service meals rose 3.9 percent over the
same period. 

Energy

The energy index fell 2.0 percent in June, as it did in May. The gasoline index decreased 3.8 percent in
June. (Before seasonal adjustment, gasoline prices fell 3.9 percent in June.) The electricity index
decreased 0.7 percent over the month and the fuel oil index decreased 2.4 percent. The index for natural
gas rose 2.4 percent in June.

The energy index increased 1.0 percent over the past 12 months. The gasoline index fell 2.5 percent over
this 12-month span. The index for electricity increased 4.4 percent over the last 12 months and the index
for natural gas rose 3.7 percent. The index for fuel oil rose 0.8 percent over the same period. 

All items less food and energy

The index for all items less food and energy rose 0.1 percent in June, the smallest increase in this index
since August 2021. The shelter index increased 0.2 percent in June. The index for rent rose 0.3 percent
over the month, as did the index for owners' equivalent rent; these were also the smallest increases in
these indexes since August 2021. The lodging away from home index decreased 2.0 percent in June, after
falling 0.1 percent in May. 

The medical care index rose 0.2 percent in June after rising 0.5 percent in May. The index for physicians'
services rose 0.1 percent over the month, as did the index for hospital services. The prescription drugs
index was unchanged in June.

The motor vehicle insurance index rose 0.9 percent in June, following a 0.1-percent decrease in May. The
index for household furnishings and operations increased 0.5 percent over the month. The indexes for
personal care, education, recreation, and apparel also increased in June.

The index for airline fares fell 5.0 percent in June, following a 3.6-percent decrease in May. Over the
month, the used cars and trucks index fell 1.5 percent, the communication index decreased 0.2 percent,
and the new vehicles index declined 0.2 percent. 

The index for all items less food and energy rose 3.3 percent over the past 12 months. The shelter index
increased 5.2 percent over the last year, accounting for nearly seventy percent of the total 12-month
increase in the all items less food and energy index. Other indexes with notable increases over the last
year include motor vehicle insurance (+19.5 percent), medical care (+3.3 percent), personal care (+3.2
percent), and recreation (+1.3 percent).

Not seasonally adjusted CPI measures

The Consumer Price Index for All Urban Consumers (CPI-U) increased 3.0 percent over the last 12 months
to an index level of 314.175 (1982-84=100). For the month, the index was unchanged prior to seasonal
adjustment.  

The Consumer Price Index for Urban Wage Earners and Clerical Workers (CPI-W) increased 2.9 percent over
the last 12 months to an index level of 308.054 (1982-84=100). For the month, the index was unchanged
prior to seasonal adjustment.  

The Chained Consumer Price Index for All Urban Consumers (C-CPI-U) increased 2.8 percent over the last 12
months. For the month, the index was unchanged on a not seasonally adjusted basis. Please note that the
indexes for the past 10 to 12 months are subject to revision. 
_______________
The Consumer Price Index for July 2024 is scheduled to be released on Wednesday, August 14, 2024, at 8:30 a.m. (ET).


----------------------------------------------------------------------------------------------------
			Notice on June 2024 CPI new vehicles index
			
Because of a disruption to software systems that affected automotive dealers across the country,
the June 2024 new vehicles index was estimated with fewer observed transaction prices than in
previous months. Despite the reduced number of observations, the new vehicles index meets CPI
publication criteria. BLS has no further information on when typical data flows will resume and is
monitoring the situation closely. 
----------------------------------------------------------------------------------------------------


Technical Note
"""

In [36]:
relevant_paragraph = extract_paragraph_based_on_index(full_text)
relevant_paragraph

'The index for gasoline fell 3.8 percent in June, after declining 3.6 percent in May, more than offsetting an increase in shelter. The energy index fell 2.0 percent over the month, as it did the preceding month. The index for food increased 0.2 percent in June. The food away from home index rose 0.4 percent over the month, while the food at home index increased 0.1 percent.'